In the quantum teleportation protocol, party A will teleport a qubit state of qubit q to party B by utilizing quantum teleportation. Party A and B each have one qubit of an entangled bell pair. A first entangles their bell pair with the q, using a cnot gate with the bell pair being the target. They then do a hadmard gate on q. After that, they measure both q and their bell pair. A then classically sends the results of both measurements to B. B will then perform 0-2 quantum gates on their bell pair to get it back to q's original state based on the measurement results. If A's bell pair measured 1, B will do an X gate on their bell pair. If q measured 1, B will do a Z gate on their bell pair. B now has q's original state in their bell pair. A bell pair that both parties have one half of is needed for this to be possible.

In [ ]:
!pip install qiskit

     |████████████████████████████████| 8.5MB 3.8MB/s 
     |████████████████████████████████| 17.6MB 250kB/s 
     |████████████████████████████████| 204kB 45.2MB/s 
     |████████████████████████████████| 204kB 46.5MB/s 
     |████████████████████████████████| 2.1MB 15.4MB/s 
     |████████████████████████████████| 51kB 5.6MB/s 
     |████████████████████████████████| 1.0MB 34.2MB/s 
     |████████████████████████████████| 194kB 26.0MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 583kB 45.1MB/s 
     |████████████████████████████████| 3.2MB 27.2MB/s 
     |████████████████████████████████| 5.5MB 31.7MB/s 
  Created wheel for qiskit: filename=qiskit-0.24.0-cp37-none-any.whl size=2918 sha256=2576c26bdeb464f624cb55bee11515423196e65bfc56a7ff9ea919e86e8ab3b4
  Stored in directory: /root/.cache/pip/wheels/22/fe/0a/26b7a409d1a9c3698b0319c7d009a97bffaae392d755158dca
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2

In [ ]:
from cmath import isclose
from math import sqrt

from qiskit import QuantumCircuit, Aer, execute
from qiskit.extensions import Initialize
from qiskit.visualization import plot_histogram

In [ ]:
class Node:
    
    def __init__(self, psi = None):
        self.psi = psi
        
    def __str__(self):
        return str(self.psi)

In [ ]:
class QuantumInternet:
    
    def __init__(self, nodes):
        self.__nodes = nodes
        self.__qc = None
    
    def __gen_bell_pair(self, ctrl, target):
        self.__qc.h(ctrl)
        self.__qc.cx(ctrl, target)

    def __sim(self):
        backend_sim = Aer.get_backend("statevector_simulator")
        sim = execute(self.__qc, backend_sim)
        return sim.result()
        
    def size(self):
        return len(self.__nodes);
    
    def teleport(self, sender, receiver):
        if sender < len(self.__nodes) and receiver < len(self.__nodes):
            data = 0
            src = 1
            dst = 2

            self.__qc = QuantumCircuit(3, 3)
            init = Initialize(self.__nodes[sender].psi)
            init.label = "init"
            self.__qc.append(init, [data])
            self.__gen_bell_pair(1, 2)
            
            self.__qc.cx(data, src)
            self.__qc.h(data)

            self.__qc.measure(data, data)
            self.__qc.measure(src, src)

            self.__qc.cx(src, dst)
            self.__qc.cz(data, dst)

            state = self.__sim().get_statevector()
            alpha_index = 0
            alpha = 0
            
            # index between statevector indices with dst measured as 0 and the indices with it measured as 1
            state_border = int((2 ** self.__qc.num_qubits) / 2)
            for i in range(state_border):
                if abs(state[i]) > abs(alpha):
                    alpha = state[i]
                    alpha_index = i
                    
            beta = max(state[state_border : ], key = abs)
            # set receiver node's psi to the outcome
            self.__nodes[receiver].psi = [alpha, beta] 
            
            sender_psi_0 = (alpha_index % 2 == 0)
            # set sender node's psi to the measurement outcome of their previous psi
            self.__nodes[sender].psi = [int(sender_psi_0), int(not sender_psi_0)]  
        else:
            print("Index %d or %d out of bounds of length %u" % (sender, receiver, len(self.__nodes)))

In [ ]:
init_psi = [1j / sqrt(3), -sqrt(2 / 3)]
print("initial: %s" % init_psi)

a = Node(init_psi)
b = Node()
c = Node()
d = Node()

internet = QuantumInternet([a, b, c, d])
for i in range(internet.size() - 1):
    internet.teleport(i, i + 1)

print("a: %s" % a)
print("b: %s" % b)
print("c: %s" % c)
print("d: %s" % d)

success = True
for i in range(len(d.psi)):
    if not isclose(d.psi[i], init_psi[i]):
        success = False
        break
print("Qubit state successfully teleported across internet: %r" % success)

initial: [0.5773502691896258j, -0.816496580927726]
a: [1, 0]
b: [0, 1]
c: [0, 1]
d: [0.5773502691896257j, (-0.8164965809277259+2.999759773043692e-16j)]
Qubit state successfully teleported across internet: True


    Chinese scientists used a satellite to beam entangled pairs of photons to 3 ground stations in China that are more than 1200 km apart. Other approaches to quantum communication include ground-based ones such as sending pairs of entangled photons through fiber-obtic cables and through open air. However, these methods are limited to a few hundred km because the photons collide with atoms along the way and their state, which is very delicate, gets disrupted. Another approach involves chaining complex devices called a Quantum Repeaters, and could possibly receive, store, and resend quantum information across a long distance. However, this method is incredibly complex and is very theoretical because of its difficulty, as it involves receiving two photons from different places, storing the quantum information, and interfering them with each other prior to sending signals on a network. This must by confirming that you have stored the photons without measuring them. The current solution that he Chinese used is beaming quantum information through space and distributing them across 10s/100s of km from the ground. The satellite has numerous crystals and lasers that generate, split, and send entangled photon pairs through separate beams to the ground. The satellite produced around 6 million entangled pairs per second, but only one out of these 6 million reached the detectors on the ground because the beams weakened while travelling through the atmosphere. However, this is still a trillion times more efficient than the transmission rate of the best telecommunication fibers. 

    More recently, last december scientists from Fermilab, a labaratoiy  the US Departement of Energy office, successfully teleported photon qubits across 44 km with a success rate of above 90%. They teleported the qubits with a fiber-optic netweork and extremely high quality single-photon detectors. They are planning on contructing a Chicago-area metropolitan quantum network in the future.

Sources: 
- https://www.scientificamerican.com/article/china-shatters-ldquo-spooky-action-at-a-distance-rdquo-record-preps-for-quantum-internet/
- https://news.fnal.gov/2020/12/fermilab-and-partners-achieve-sustained-high-fidelity-quantum-teleportation/